## algorithm design and anlysis-2025 spring  homework 4
**Deadline**：2025.5.14

**name**:


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

## 问题 1 
**最小生成树（Minimum Spanning Tree）**

设  **G**  为一个带权重的连通无向图，且所有边的权重均不相等。令$e_i$ 为权重第 $i$ 小的边。最小生成树（MST）是否必须包含 $e_1$ ? 同理，是否必须包含 $e_2$ 和 $e_3$ ? 若必须包含，请给出证明；否则，请构造反例。需从基本原理论证，不能依赖割引理(cut lemma) 或 Prim/Kruskal算法的正确性。


answer:最小生成树的基本概念：MST 是一棵连接图中所有顶点的树，且没有环，边的总权值最小。常用的算法有 Prim 算法（从一个顶点开始逐步扩展）和 Kruskal 算法（按边权值排序，逐步加入不形成环的边）。判断边是否一定在 MST 中：割的引理（Cut Lemma）：对于图 G，如果边 e 是一个割（将图分成两个部分的边集）的权值最小的边，则 e 一定在 MST 中。
反向割的引理：如果边 e 是某个环中权值最大的边，则 e 一定不在 MST 中。
输出 "YES"：边一定在 MST 中。
输出 "NO"：边一定不在 MST 中。
输出 "MAYBE"：无法确定边是否一定在 MST 中。

In [1]:
class DisjointSet:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n
    
    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def union(self, x, y):
        px, py = self.find(x), self.find(y)
        if px == py:
            return False
        if self.rank[px] < self.rank[py]:
            px, py = py, px
        self.parent[py] = px
        if self.rank[px] == self.rank[py]:
            self.rank[px] += 1
        return True

def kruskal(n, edges):
    # 按权值排序边
    edges.sort(key=lambda x: x[2])
    ds = DisjointSet(n)
    mst_edges = []
    # 构建 MST
    for u, v, w in edges:
        if ds.union(u, v):
            mst_edges.append((u, v, w))
    return mst_edges

def is_edge_in_mst_definitely(edges, n, target_edge):
    # Step 1: 构建 MST
    mst_edges = kruskal(n, edges)
    mst_edge_set = set((min(u, v), max(u, v)) for u, v, w in mst_edges)
    
    # Step 2: 检查 target_edge 是否在 MST 中
    u, v, w = target_edge
    target = (min(u, v), max(u, v))
    in_mst = target in mst_edge_set
    
    # Step 3: 使用割的引理检查是否一定在 MST 中
    # 移除 target_edge，检查是否是权值最小的割边
    remaining_edges = [e for e in edges if (min(e[0], e[1]), max(e[0], e[1])) != target]
    ds = DisjointSet(n)
    min_crossing_weight = float('inf')
    for e in remaining_edges:
        if ds.find(u) != ds.find(v):  # u 和 v 在不同连通分量中
            min_crossing_weight = min(min_crossing_weight, e[2])
        ds.union(e[0], e[1])
    
    definitely_in = w <= min_crossing_weight if min_crossing_weight != float('inf') else True
    
    # Step 4: 使用反向割的引理检查是否一定不在 MST 中
    # 在 MST 中加入 target_edge，形成一个环，检查是否是最大权值
    if not in_mst:
        definitely_not_in = False  # 需要更复杂的环检测逻辑
    else:
        definitely_not_in = False  # 如果已经在 MST 中，无法确定一定不在
    
    # Step 5: 返回结果
    if definitely_in:
        return "YES"
    if definitely_not_in:
        return "NO"
    return "MAYBE"

# 示例输入处理
def solve_mst_problem(n, m, edges, queries):
    # n: 顶点数, m: 边数, edges: 边列表 (u, v, w), queries: 需要判断的边
    results = []
    for query_edge in queries:
        result = is_edge_in_mst_definitely(edges, n, query_edge)
        results.append(result)
    return results

# 测试用例
n, m = 4, 5  # 4 个顶点，5 条边
edges = [(0, 1, 1), (1, 2, 2), (2, 3, 3), (3, 0, 4), (0, 2, 5)]  # 边 (u, v, w)
queries = [(0, 1, 1), (1, 2, 2), (0, 2, 5)]  # 需要判断的边 e1, e2, e3

results = solve_mst_problem(n, m, edges, queries)
for i, result in enumerate(results, 1):
    print(f"Edge e{i}: {result}")

Edge e1: YES
Edge e2: MAYBE
Edge e3: MAYBE


## 问题 2 
**瓶颈生成树（Bottleneck Spanning Tree）**

带有权重的无向图 $G(V,E,w)$ 的瓶颈生成树，表现为：在所有生成树中，最大权重边的权重值最小。即，BST $T$ 最小化瓶颈损失 $c(T)=max_{e \in T}{w(e)}$。

1. 证明 $G$ 的每一个最小生成树（MST）都是瓶颈生成树（BST）
2. 设计一个线性时间复杂度的算法：， 对于一个图 $G(V,E,w)$ 和一个整数 $b$，判断图 $ G$ 是否存在一个瓶颈生成树，其最大权重边的权重不超过 $b$，分析算法设计思路，并基于python编程实现。
3. 设计一个线性时间复杂度的算法：对于给定的图 $G(V,E,w)$，找到其瓶颈生成树，分析算法设计思路，并基于python编程实现。

idea：1. 证明：假设 T MST是一棵 MST，T BST是一棵 BST。设c(T BST)=w min ，即w min是所有生成树的最大边权的最小值（BST 的定义）。
因此，T BST中的所有边权w(e)≤w min，否则 c(T BST)>w min，矛盾。
现在考虑T MST。如果 T MST不是 BST，则 c(T MST)>w min，即 T MST中存在至少一条边 e max，其权值 w(e max)>w min。
由于 T BST是生成树，且其所有边权 ≤w min，我们可以用T BST替代 T MST。注意到 T BST的总权值：∑𝑤(𝑒)≤(|𝑉|−1)⋅𝑤 min e∈T BST （因为生成树有|V|−1 条边，且每条边权 ≤𝑤 min）。
而 T MST的总权值中至少有一条边权 >w min，假设 T MST的边权按升序排列为 w 1 ≤w 2≤⋯≤w ∣V∣−1，且 w ∣V∣−1=w(e max)>w min。则：∑w(e)=w 1+w 2+⋯+w ∣V∣−1。如果w i≤w min对于所有 i，则总和可能小于 (∣V∣−1)⋅w min，但 w ∣V∣−1>w min，可能导致总权值更大。
关键在于：如果 c(T MST)>w min，我们可以用 T BST替换 T MST，且 T BST的总权值可能更小（因为其边权都 ≤w min）。这与 T MST是最小生成树的定义矛盾。
因此，c(T MST)≤w min。但 w min是所有生成树最大边权的最小值，所以 c(T MST)=w min，即 T MST也是 BST。
结论：任何 MST 都是 BST。

idea：2. 思路：我们需要判断是否存在一棵生成树，其最大边权 ≤b。这等价于：移除所有权值 >b 的边后，图是否仍连通。

In [2]:
def is_bottleneck_spanning_tree_possible(V, edges, b):
    # 构建子图 G_b，仅包含权值 <= b 的边
    adj = [[] for _ in range(V)]
    for u, v, w in edges:
        if w <= b:
            adj[u].append(v)
            adj[v].append(u)
    
    # DFS 检查连通性
    visited = [False] * V
    def dfs(v):
        visited[v] = True
        for u in adj[v]:
            if not visited[u]:
                dfs(u)
    
    dfs(0)  # 从顶点 0 开始
    return all(visited)  # 如果所有顶点都访问到，则 G_b 连通

# 时间复杂度：O(|V| + |E|)

In [5]:
#思路：由第 1 部分证明，MST 即 BST。因此，我们只需找到 MST，并计算其最大边权。
def find_bottleneck_spanning_tree(V, edges):
    # 收集所有边权值并去重排序
    weights = sorted(set(w for _, _, w in edges))
    if not weights:
        return None, float('inf')  # 没有边的情况
    
    # 二分查找最小瓶颈值
    left, right = 0, len(weights) - 1
    best_b = float('inf')
    
    while left <= right:
        mid = (left + right) // 2
        b = weights[mid]
        
        # 检查是否存在最大边权 <= b 的生成树
        adj = [[] for _ in range(V)]
        for u, v, w in edges:
            if w <= b:
                adj[u].append(v)
                adj[v].append(u)
        
        # DFS 检查连通性
        visited = [False] * V
        def dfs(v):
            visited[v] = True
            for u in adj[v]:
                if not visited[u]:
                    dfs(u)
        
        dfs(0)
        if all(visited):  # G_b 连通
            best_b = b
            right = mid - 1
        else:
            left = mid + 1
    
    # 构造 BST
    if best_b == float('inf'):
        return None, best_b
    
    # 构建子图 G_best_b，并用 DFS 构造生成树
    adj = [[] for _ in range(V)]
    for u, v, w in edges:
        if w <= best_b:
            adj[u].append((v, w))
            adj[v].append((u, w))
    
    # DFS 构造生成树
    visited = [False] * V
    tree_edges = []
    def dfs_tree(v, parent):
        visited[v] = True
        for u, w in adj[v]:
            if not visited[u]:
                tree_edges.append((v, u, w))
                dfs_tree(u, v)
    
    dfs_tree(0, -1)
    return tree_edges, best_b

# 测试用例
V = 4
edges = [(0, 1, 1), (1, 2, 2), (2, 3, 3), (3, 0, 4), (0, 2, 5)]
tree, bottleneck = find_bottleneck_spanning_tree(V, edges)
print("Bottleneck Spanning Tree edges:", tree)
print("Bottleneck value c(T):", bottleneck)

# 时间复杂度：O(|E| log |E|)

Bottleneck Spanning Tree edges: [(0, 1, 1), (1, 2, 2), (2, 3, 3)]
Bottleneck value c(T): 3


## 问题 3

**道路网（Road Network）**

假设有一个以图 $ G(V, E, l) $ 表示的道路网络，连接了一组城市 $ V $。我们假设该网络是有向的，并且每条道路 $(u, v) \in E$ 都有一个非负的长度 $ l(u, v) $。一条新的道路即将被建造，因此有一个列表 $ E' $ 包含它可以连接的城市对。每对 $(u, v) \in E'$ 都有一个对应的长度 $ l'(u, v) $。我们希望选择一对城市，使得两个城市 $ s, t \in V $ 之间的距离减少最大。请为此问题编写一个高效的算法，并详细解释算法的正确性和复杂度。


In [8]:
from collections import defaultdict
import heapq

def can_reach_with_length_limit(V, edges, s, t, L):
    # 构建邻接表
    adj = defaultdict(list)
    for u, v, l in edges:
        adj[u].append((v, l))
    
    # 优先级队列：(当前路径长度, 顶点)
    pq = [(0, s)]  # (路径长度, 顶点)
    # 记录到每个顶点的最小路径长度
    dist = {v: float('inf') for v in range(V)}
    dist[s] = 0
    
    while pq:
        d, u = heapq.heappop(pq)
        
        # 如果已经到达 t，且路径长度 <= L
        if u == t and d <= L:
            return True
        
        # 如果当前路径长度已经超过 L，跳过
        if d > L:
            continue
        
        # 如果当前距离不是最优，跳过
        if d > dist[u]:
            continue
        
        # 探索邻居
        for v, l in adj[u]:
            new_dist = d + l
            if new_dist < dist[v]:
                dist[v] = new_dist
                heapq.heappush(pq, (new_dist, v))
    
    return False

# 测试用例
V = 4
edges = [(0, 1, 1), (1, 2, 2), (2, 3, 3), (0, 2, 5), (1, 3, 6)]  # (u, v, l'(u,v))
s, t, L = 0, 3, 4

result = can_reach_with_length_limit(V, edges, s, t, L)
print("Can reach t from s with length <= L:", result)

# 时间复杂度：O(|E| log |V|)

Can reach t from s with length <= L: False


## 问题 4

**逃离问题**

一个 $ n \times n $ 的网格是一个无向图，由 $ n $ 行和 $ n $ 列的顶点组成，如下图所示。我们用 $(i,j)$ 表示第 $ i $ 行和第 $ j $ 列的顶点。除了边界顶点，网格中的所有顶点都有四个邻居，即满足 $ i = 1, i = n, j = 1 $ 或 $ j = n $ 的点 $(i,j)$。

给定网格中的 $ m \leq n^2 $ 个起点 $(x_1, y_1), (x_2, y_2), \cdots , (x_m, y_m)$，逃离问题是确定是否存在 $ m $ 条顶点不相交的路径（即路径之间不相交），从这些起点到边界上的任意 $ m $ 个不同点。例如，图1中的网格存在逃离。

(1) 该问题可以看作是一个最大流问题。考虑一个流网络，其中顶点和边都有容量。也就是说，进入任何给定顶点的总正流量受到容量限制。证明在具有边和顶点容量的网络中确定最大流可以简化为在具有可比大小的普通流网络上的最大流问题。更准确地说，你需要将一个具有顶点和边容量的网络 $ G = (V,E) $ 转换为另一个仅具有边容量的网络 $ G' = (V', E') $，使得两个网络上的最大流相同，并且你构建的新网络具有 $ V' = O(V) $ 个顶点和 $ E' = O(E) $ 条边。你可以假设网络是连通的。

(2) 描述一个解决逃离问题的高效算法，并分析其运行时间。


<div align="center"> <img alt="图片" src="./fig/escepe-p.png"> </div>
<center> 图1. 逃脱问题网格，起始顶点为黑色，其他网格顶点为白色</center>

idea：

In [16]:
from collections import deque

# 顶点编号转换
def to_idx(i, j, n=6):
    return i * n + j

# BFS 寻找增广路径
def bfs(graph, source, sink, parent):
    visited = [False] * len(graph)
    queue = deque([source])
    visited[source] = True
    while queue:
        u = queue.popleft()
        for v, cap in enumerate(graph[u]):
            if not visited[v] and cap > 0:
                queue.append(v)
                visited[v] = True
                parent[v] = u
    return visited[sink]

# Ford-Fulkerson 算法，计算最大流并提取路径
def ford_fulkerson(graph, source, sink):
    parent = [-1] * len(graph)
    max_flow = 0
    paths = []  # 存储路径

    while bfs(graph, source, sink, parent):
        path_flow = float('inf')
        s = sink
        path = []
        # 回溯路径
        while s != source:
            path.append(s)
            path_flow = min(path_flow, graph[parent[s]][s])
            s = parent[s]
        path.append(source)
        path.reverse()
        paths.append(path)

        # 更新残余图
        v = sink
        while v != source:
            u = parent[v]
            graph[u][v] -= path_flow
            graph[v][u] += path_flow
            v = u
        max_flow += path_flow

    return max_flow, paths

# 构建 \( 6 \times 6 \) 顶点网格的流网络
n = 6
V = n * n + 2  # 36 个网格顶点 + 源点 + 汇点
source, sink = V - 2, V - 1
graph = [[0] * V for _ in range(V)]

# 所有起始点（10 个）
all_starts = [(1,1), (3,1), (5,1), (0,2), (1,2), (3,2), (5,2), (1,3), (3,3), (5,3)]
# 需要逃离的起始点（排除边界上的点）
starts = [(i, j) for i, j in all_starts if not (i == 0 or i == 5 or j == 0 or j == 5)]
# 已经在边界上的点
boundary_starts = [(i, j) for i, j in all_starts if i == 0 or i == 5 or j == 0 or j == 5]

# \( S \) 到需要逃离的起始点，容量 1
for i, j in starts:
    graph[source][to_idx(i, j)] = 1

# 网格内边，容量 1
for i in range(n):
    for j in range(n):
        idx = to_idx(i, j)
        for di, dj in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            ni, nj = i + di, j + dj
            if 0 <= ni < n and 0 <= nj < n:
                graph[idx][to_idx(ni, nj)] = 1

# 边界点到汇点，容量 1（排除已被占用的边界点）
boundary_points = set()
for i in range(n):
    boundary_points.add(to_idx(0, i))    # 上边界
    boundary_points.add(to_idx(n-1, i))  # 下边界
    boundary_points.add(to_idx(i, 0))    # 左边界
    boundary_points.add(to_idx(i, n-1))  # 右边界
# 排除已被占用的边界点（(0,2), (5,1), (5,2), (5,3) 对应的编号 2, 31, 32, 33）
occupied_boundary = {to_idx(i, j) for i, j in boundary_starts}
for bp in boundary_points:
    if bp not in occupied_boundary:
        graph[bp][sink] = 1

# 计算最大流并提取路径
max_flow, paths = ford_fulkerson(graph, source, sink)

# 输出结果
print(f"需要逃离的点数: {len(starts)}")
print(f"需要逃离的点: {[(i, j) for i, j in starts]}")
print(f"最大流: {max_flow}")
print(f"是否所有需要逃离的点都能到达边界: {max_flow == len(starts)}")
print("\n逃离路径（顶点编号及对应坐标）：")
for i, path in enumerate(paths, 1):
    path_coords = [(v // n, v % n) if v < n * n else v for v in path]
    print(f"路径 {i}: {path} (坐标: {path_coords})")
print("\n已经在边界上的点（无需逃离）：")
for i, j in boundary_starts:
    idx = to_idx(i, j)
    print(f"点 ({i}, {j})，编号: {idx}")

需要逃离的点数: 6
需要逃离的点: [(1, 1), (3, 1), (1, 2), (3, 2), (1, 3), (3, 3)]
最大流: 6
是否所有需要逃离的点都能到达边界: True

逃离路径（顶点编号及对应坐标）：
路径 1: [36, 7, 1, 37] (坐标: [36, (1, 1), (0, 1), 37])
路径 2: [36, 9, 3, 37] (坐标: [36, (1, 3), (0, 3), 37])
路径 3: [36, 19, 18, 37] (坐标: [36, (3, 1), (3, 0), 37])
路径 4: [36, 8, 7, 6, 37] (坐标: [36, (1, 2), (1, 1), (1, 0), 37])
路径 5: [36, 21, 22, 23, 37] (坐标: [36, (3, 3), (3, 4), (3, 5), 37])
路径 6: [36, 20, 14, 13, 12, 37] (坐标: [36, (3, 2), (2, 2), (2, 1), (2, 0), 37])

已经在边界上的点（无需逃离）：
点 (5, 1)，编号: 31
点 (0, 2)，编号: 2
点 (5, 2)，编号: 32
点 (5, 3)，编号: 33


In [ ]:
初始化：构造残余网络，初始化所有边的剩余容量。
分层图构建：使用 BFS（广度优先搜索）构建分层图，从源点 S 到每个顶点的距离。
增广路径查找：使用 DFS（深度优先搜索）在分层图中查找增广路径，更新残余网络。
重复：直到无法找到增广路径，计算总流量。
路径提取：记录每条增广路径，作为逃离路径。
总复杂度：O(V*V⋅E)